In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split


In [2]:
def make_dataset(path):
    images=[]
    angry=0
    sad=0
    happy=0
    other=0
    for filename in os.listdir(path):
        new_path=os.path.join(path,filename)
        for img in os.listdir(new_path):
            image=cv2.imread(os.path.join(new_path,img))
            image=cv2.resize(image,(224,224))
            if image is not None:
                if filename=="Angry":
                    images.append(image)
                    angry+=1
                elif filename=="Sad":
                    images.append(image)
                    sad+=1
                elif filename=="happy":
                    images.append(image)
                    happy+=1
    Angry=np.zeros(angry)
    Sad=np.ones(sad)
    Happy=np.repeat(2,happy)
    label=np.hstack((Angry,Sad,Happy))
    label=tf.keras.utils.to_categorical(label,num_classes=3)
    return images,label

In [3]:
images,label=make_dataset("Master Folder")
images=np.array(images)
print(images.shape,label.shape)

(812, 224, 224, 3) (812, 3)


In [4]:
train_X,trail_X,train_label,trail_label=train_test_split(images,label,test_size=0.3,random_state=42,shuffle=True,stratify=label)
validation_X,test_X,validation_label,test_label=train_test_split(images,label,test_size=0.7,random_state=42,shuffle=True,stratify=label)

In [5]:
def preprocess_image(images):
    image=tf.cast(images,tf.float32)/255.0
    return image

In [6]:
train_X=preprocess_image(train_X)
validation_X=preprocess_image(validation_X)
test_X=preprocess_image(test_X)

In [7]:
callbacks=[EarlyStopping(monitor = "val_loss", patience = 10, 
            restore_best_weights = True, mode='min')
            ,ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, mode='min')
            ,ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
            ]

In [45]:
def model_build(inputs):
    
    # x=tf.keras.layers.Conv2D(16,activation='relu',kernel_size=3,input_shape=(224,224,3))(inputs)
    # x=tf.keras.layers.AveragePooling2D((2,2))(x)
    
    # x=tf.keras.layers.Conv2D(32,activation='relu',kernel_size=3)(x)
    # x=tf.keras.layers.AveragePooling2D((2,2))(x)
    
    # x=tf.keras.layers.Conv2D(64,activation='relu',kernel_size=3)(x)
    # x=tf.keras.layers.AveragePooling2D((6,6))(x)

    feature_extract=tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224,224,3),weights="imagenet",include_top=False)
    feature_extract.trainable=False
    x=feature_extract(inputs,training=False)
    x=tf.keras.layers.AveragePooling2D((3,3))(x)
    x=tf.keras.layers.Flatten()(x)
    x=tf.keras.layers.Dropout(0.5)(x)
    x=tf.keras.layers.Dense(64,activation='relu')(x)
    x=tf.keras.layers.Dropout(0.5)(x)
    x=tf.keras.layers.Dense(3,activation='sigmoid')(x)
    
    return x

In [46]:
def define_compile_model():
    inputs=tf.keras.layers.Input(shape=(224,224,3))
    output=model_build(inputs)
    model=tf.keras.Model(inputs=inputs,outputs=output)
    model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    return model

In [47]:
model=define_compile_model()
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 average_pooling2d_17 (Aver  (None, 2, 2, 1280)        0         
 agePooling2D)                                                   
                                                                 
 flatten_8 (Flatten)         (None, 5120)              0         
                                                                 
 dropout_21 (Dropout)        (None, 5120)              0         
                                                                 
 dense_21 (Dense)            (None, 64)                3277

In [48]:
epochs=50
batch_size=16
history = model.fit(train_X, train_label,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(validation_X, validation_label),callbacks=callbacks)


Epoch 1/50
36/36 [==============================] - 32s 779ms/step - loss: 1.5731 - accuracy: 0.4894 - val_loss: 0.8102 - val_accuracy: 0.6008 - lr: 0.0010
Epoch 2/50
36/36 [==============================] - 27s 742ms/step - loss: 0.8795 - accuracy: 0.5863 - val_loss: 0.6521 - val_accuracy: 0.7572 - lr: 0.0010
Epoch 3/50
36/36 [==============================] - 26s 725ms/step - loss: 0.7680 - accuracy: 0.6514 - val_loss: 0.5150 - val_accuracy: 0.8148 - lr: 0.0010
Epoch 4/50
36/36 [==============================] - 26s 725ms/step - loss: 0.6867 - accuracy: 0.7113 - val_loss: 0.4519 - val_accuracy: 0.8519 - lr: 0.0010
Epoch 5/50
36/36 [==============================] - 27s 758ms/step - loss: 0.6103 - accuracy: 0.7500 - val_loss: 0.3504 - val_accuracy: 0.8765 - lr: 0.0010
Epoch 6/50
36/36 [==============================] - 26s 741ms/step - loss: 0.5275 - accuracy: 0.7465 - val_loss: 0.2838 - val_accuracy: 0.9218 - lr: 0.0010
Epoch 7/50
36/36 [==============================] - 26s 726ms/st

c:\Python\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


36/36 [==============================] - 26s 726ms/step - loss: 0.1152 - accuracy: 0.9507 - val_loss: 0.0036 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 36/50
36/36 [==============================] - 27s 743ms/step - loss: 0.1493 - accuracy: 0.9349 - val_loss: 0.0038 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 37/50
36/36 [==============================] - 25s 711ms/step - loss: 0.1099 - accuracy: 0.9648 - val_loss: 0.0043 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 38/50
36/36 [==============================] - 26s 726ms/step - loss: 0.0906 - accuracy: 0.9665 - val_loss: 0.0029 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 39/50
36/36 [==============================] - 27s 742ms/step - loss: 0.1028 - accuracy: 0.9542 - val_loss: 0.0028 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 40/50
36/36 [==============================] - 27s 742ms/step - loss: 0.1074 - accuracy: 0.9560 - val_loss: 0.0029 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 41/50
36/36 [==============================] - 26s 725ms/step - 

In [202]:
model.evaluate(train_X,train_label)

18/18 [==============================] - 18s 1s/step - loss: 0.0171 - accuracy: 0.9982


[0.01714949496090412, 0.9982394576072693]

In [49]:
model.evaluate(test_X,test_label)

18/18 [==============================] - 19s 1s/step - loss: 0.3564 - accuracy: 0.9297


[0.3563951551914215, 0.9297012090682983]